In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
import healpy as hp
import mastcasjobs
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
import pyvo
import requests
from MyToolkit import *
import time
import os

In [ ]:
### Here we are attempting to obtain the PS1-ELQS effective coverage area using **healpy** (HEALpix; Gorski et al. 2005) tools

In [ ]:
effective_area = healpix_sky_map_ps1_cover_area(ra_col_schind, dec_col_schind, 6)
print("Effective area covered by the data is {:.3f} deg^2".format(effective_area))

##### Above we included only the 592 quasars in the HEALpix estimate. Now let's add all the targets from the JKW2 color cut of PS1.

In [ ]:
tap_service = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')

In [ ]:
wise_2mass_query = """SELECT
        ra, dec, j_m_2mass, k_m_2mass, w1mpro, w2mpro, w1snr, w2snr,
        glat
        FROM allwise_p3as_psd
        WHERE k_m_2mass - w2mpro >= 1.8 - 0.848 * (j_m_2mass - k_m_2mass) AND
        glat NOT BETWEEN -20 AND 20 AND
        j_m_2mass > 0 AND
        w1snr >= 5 AND
        w2snr >= 5
        """

wise_2mass_jkw2_colorcut = tap_service.submit_job(wise_2mass_query)
wise_2mass_jkw2_colorcut.run()

In [ ]:
go = 1
i = 1
while go == 1:
    time.sleep(30)
    if wise_2mass_jkw2_colorcut.phase in ('COMPLETED'):
        result = wise_2mass_jkw2_colorcut.fetch_result()
        table = result.to_table()
        table.write(DP2_DIRECTORY + "Data/wise_2mass_jkw2_colorcut_full.csv")
        go = 0
    i += 1

In [ ]:
go = 1
i = 1
while go == 1:
    time.sleep(30)
    print(i),
    if wise_2mass_jkw2_colorcut.phase in ('COMPLETED'):
        URL = wise_2mass_jkw2_colorcut.result_uri

        response = requests.get(URL)
        open(DP2_DIRECTORY + "Data/wise_2mass_jkw2_colorcut_full.xml", "wb").write(response.content)
        go = 0
    i += 1

In [ ]:
query = """SELECT	
        m.ra, m.dec,	
        o.ObjID as PS1_ObjID,x.ra as ps_ra,x.dec as ps_dec,	
        o.gMeanPSFMag, o.gMeanPSFMagErr, o.gMeanKronMag, o.gMeanApMag,	
        o.rMeanPSFMag, o.rMeanPSFMagErr, o.rMeanKronMag, o.rMeanApMag,	
        o.iMeanPSFMag, o.iMeanPSFMagErr, o.iMeanKronMag, o.iMeanApMag,	
        o.zMeanPSFMag, o.zMeanPSFMagErr, o.zMeanKronMag, o.zMeanApMag,	
        o.yMeanPSFMag, o.yMeanPSFMagErr, o.yMeanKronMag, o.yMeanApMag,	
        o.gMeanPSFmagNpt,o.rMeanPSFmagNpt,o.iMeanPSFmagNpt,o.zMeanPSFmagNpt,o.yMeanPSFmagNpt,	
        o.gFlags, o.gQfPerfect,	
        o.rFlags, o.rQfPerfect,	
        o.iFlags, o.iQfPerfect,	
                
        o.zFlags, o.zQfPerfect,	
        o.yFlags, o.yQfPerfect,	
        sp.gpetRadius,sp.rpetRadius,sp.ipetRadius,sp.zpetRadius,sp.ypetRadius,	
        sot.iinfoFlag, sot.iinfoFlag2	
                	
        from mydb.wise_2mass_jkw2_colorcut_subsample AS m	
                
        CROSS APPLY (SELECT * FROM dbo.fGetNearestObjEq(m.ra,m.dec,0.066)) AS x	
        JOIN MeanObject o on o.ObjID = x.ObjId	
                
        LEFT JOIN StackPetrosian AS sp ON sp.objID = o.objID	
        LEFT JOIN StackObjectThin AS sot ON sot.objID = o.objID	
                
        WHERE (o.iMeanPSFMag > 0 AND o.iMeanPSFMag <= 19.0)	
        AND o.zMeanPSFMag > 0	
        AND o.yMeanPSFMag > 0	
        AND o.iQfPerfect >= 0.85 and o.zQfPerfect >= 0.85	
	
        AND (-0.3 <= iMeanPSFMag - iMeanApMag OR iMeanPSFMag - iMeanApMag <= 0.3)	

        AND (sot.iinfoFlag & 0x00000008 = 0)
        AND (sot.iinfoFlag & 0x00000010 = 0) 
        AND (sot.iinfoFlag & 0x00000020 = 0) 
        AND (sot.iinfoFlag & 0x00000080 = 0)
        AND (sot.iinfoFlag & 0x00000100 = 0) 	
        AND (sot.iinfoFlag & 0x00000400 = 0) 
        AND (sot.iinfoFlag & 0x00000800 = 0) 
        AND (sot.iinfoFlag & 0x00001000 = 0) 	
        AND (sot.iinfoFlag & 0x00002000 = 0)	
        AND (sot.iinfoFlag & 0x00008000 = 0) 
        AND (sot.iinfoFlag & 0x00010000 = 0) 	
        AND (sot.iinfoFlag & 0x00020000 = 0) 
        AND (sot.iinfoFlag & 0x00040000 = 0) 
        AND (sot.iinfoFlag & 0x00400000 = 0) 
        AND (sot.iinfoFlag & 0x10000000 = 0) 
        AND (sot.iinfoFlag & 0x20000000 = 0)
        AND (sot.iinfoFlag & 0x40000000 = 0)
        AND (sot.iinfoFlag & 0x80000000 = 0) 
        AND (sot.iinfoFlag2 & 0x00000008 = 0) 
        AND (sot.iinfoFlag2 & 0x00000010 = 0) 	
        AND (sot.iinfoFlag2 & 0x00000020 = 0) 
        """

jobs = mastcasjobs.MastCasJobs(username = "vibin.narayanan", password = "vibinps1pwd", context = "PanSTARRS_DR1")
ps1_jkw2_cut = jobs.quick(query, task_name = "PS1 jkw2 color cut")